In [258]:
#lib para facilitar o uso de dataframes
import pandas as pd
#lib para facilitar alguns cálculos
import numpy as np
#Lib para auxiliar o processo de count
from collections import Counter

import itertools

In [259]:
#lendo o csv com o dataset pronto
df_sells = pd.read_csv('data/movies_ratings.csv')

In [257]:
#Confirmando o formato dos dados
df_sells.head()

,318,356,296,593,260,2571,527,480,1196,1,...,377,3578,1265,1240,1193,1136,1097,648,1221,1291
0,n,y,y,y,n,n,y,y,n,n,...,y,n,n,n,n,n,n,n,n,n
1,y,y,y,y,n,n,y,n,n,n,...,n,n,n,n,n,n,n,n,n,n
2,n,y,y,n,y,n,n,y,y,n,...,n,n,y,y,n,y,y,n,n,y
3,y,y,n,n,y,n,n,y,y,y,...,y,n,n,y,n,y,y,n,n,y
4,y,y,y,y,y,y,y,n,y,n,...,n,y,y,n,n,n,n,n,n,y


# Regras de associacao

Regras de associacao sao tecnicas de mineiracao de dados usadas para descobrir associacoes interessantes entre atributos de um determinado banco de dados.
A definicao classica de regras de associacao foi apresentada em Agrawal, Imielinski, & Swami (1993) and Han & Kamber, (2006)), e e definida como o seguinte:

Seja T {t1,t2...tn} um conjunto de transacoes e I {i1,i2...in} um conjunto de itens, definimos D como o dado relevante para a tarefa sendo um conjunto de transacoes onde cada T e um conjunto de itens de maneira que T seja um subconjunto de I.

Uma regra de associacao tem a forma X -> Y, onde X e Y sao conjuntos disjuntos

Em nosso dataset as transacoes T representao o conjunto de votos de cada representante, e os Itens representao a reposta ao voto para cada projeto.

O algoritmo aprior é usado para encontrar conjuntos frequentes de items em databases como o nosso.
Ele funciona com a afirmativa que:
> Um sub conjunto de um conjunto frequente provavelmente e trambem um conjunto frequente

Por exemplo:
   se {i1,i2} é um conjunto frequente entao provavelmente {i1} e {i2} tambem devem ser conjuntos de items frequentes
   
- O algoritmo faz varias iteracoes em busca de itens frequentes e usa esses itens frequentes para gerar as regras de associacao

Abaixo vemos a implementacao do algoritmo passo a passo

---

Antes de comecarmos com o algoritmo precisamos realizar a definicao das metricas

In [241]:
def get_suporte(qtd_X, qtd_total):
    '''
    suporte = Numero de transacoes que um item aparece
              ---------------------------------------
              Numero total de transacoes
    '''
    return (qtd_X*1.0)/qtd_total


## Passo 1  e  2

> Passo 1. No primeiro passo vamos criar uma tabela com todos os counts individuais de cada votacao
positiva para cada item, calculando tambem seu suporte

> Passo 2. No segundo passo vamos apenas ficar com suportes acima de um determinado threshold para dar continuidade ao algoritmo

In [242]:
SUP_THRESHOLD = 0.3
CONF_TRESHOLD = 0.70

In [243]:
def itens_distintos(df,SUP_THRESHOLD):
    '''
    1. Conta a quantidade de y e n em todas as colunas, pegando apenas y no final
    e Transpondo para melhorar o entendimento da tabela
    2. Adiciona uma coluna com o suporte calculado
    3. Passa o threshold de suporte retornando apenas os itens acima do suporte
    4. Retorna uma lista dos itens filtrados no passo 3
    '''
    #1.
    countings = pd.DataFrame(df.apply(pd.Series.value_counts).T['y'])
    #2.
    countings['suporte'] = countings.apply(lambda k: get_suporte(k['y'], df_sells.shape[0]),axis=1)
    #3.
    countings = countings.loc[countings['suporte'] >= SUP_THRESHOLD]
    return list(countings.index)

## Passos 3 e 4

> Passo 3 - a partir da lista gerada anteriormente geramos todas as combinacoes possiveis
dentro do propio set (ordem nao importando) com numero de combinacoes = K

In [244]:
def gera_combinacao(candidatos, k):
    '''
    A partir de uma lista gera todas as combinacoes possiveis de tamanho k
    Ex:
    para k = 2
    F3 = {{1, 2, 3}, {1, 2, 4}, {1, 3, 4}, {1, 3, 5}, {2, 3, 4}}.
    Alguns grupos gerados
    1,2,3 e 1,2,4
    1,2,3 e 1,3,4
    1,2,3 e 1,3,5
    ...
    '''
    #geramos todas as combinacoes possiveis de i para o resto da lista
    #usando a funcao combinations de itertools
    return [list(x) for x in itertools.combinations(candidatos, k)]


In [245]:
#candidatos = itens_distintos(df_voting)
#novos_candidatos = gera_combinacao(candidatos, k=2)

In [246]:
def merge_lists(lista):
    '''
    recebe uma lista de listas e transforma em uma lista unica para facilitar a comparacao
    '''
    merged = list(itertools.chain(*lista))
    return merged

In [247]:
def compare_sets(f1,f2):
    '''
    compara dois sets de dados, retorna True se 
    os dois sets tem exatamente os mesmos items, caso contrario false
    '''
    #vamos verificar se tem apenas um elem diferente 
    #Temos apenas um elem diferente, vamos verificar se e o ultimo
    #se o tamanho dos sets for igual e apenas o ultimo elem diferir
    if (len(f1) == len(f2)) and (list(f1)[-1] != list(f2)[-1]) and ((list(f1)[:-1] == list(f2)[:-1])):
        return True
    else:
        return False
        

In [248]:
def gera_suporte_combinacoes(df_voting,lista_combinacoes,SUP_THRESHOLD, create_rules=False):
    '''
    Essa funcao equivale as linhas 5 a 9 do slide 26 sobre regras de associacao
    Basicamente a partir de uma serie de candidatos
    itera um a um e mede o suporte, os mantendo apenas se a medicao ficar acima do threshold
    setado
    '''
    qtd_total_votos = df_voting.shape[0]
    combinacoes_uteis = []
    #crio copia do dataframe original
    for combinacao in lista_combinacoes:
        #Apenas garantindo, na primeira iteracao os tipos podem ser diferentes
        if type(combinacao) != list:
            combinacao = [combinacao]
        df = df_voting.copy()
        #Conto para cada coluna a quantidade de vezes que um voto aparece
        #usamos filtros sucessivos para isso
        for col in combinacao:
            df = df.loc[df[col]=='y']
        #Medimos o suporte
        suporte_par = get_suporte(df.shape[0],qtd_total_votos)
        ##Para criacao das regras
        if create_rules:
            return suporte_par
        if suporte_par >= SUP_THRESHOLD:
            combinacoes_uteis.append(combinacao)
    return combinacoes_uteis

In [249]:
def candidate_gen(conj):
    '''
    Codigo representado no slide 27 (Regras de Associacao)
    '''
    #inicia um conjunto vazio
    Ck = []
    #Geramos todos os pares possiveis do conjunto em uso
    #checar comentario em gera_combinacao()
    todos_os_pares = gera_combinacao(conj, 2)
    #apenas para evitar erro na primeira iteracao pois os pares nao vem como listas
    if type(conj[0]) != list:
        return todos_os_pares
    #para cada para gerado comparamos usando a funcao compare_sets
    #eh a comparacao explicada no slide 28 Passo Join
    #Realiza passo de Juncao
    for f1,f2 in todos_os_pares:
        if compare_sets(f1,f2):
            f1 = set(f1)
            f2 = set(f2)
            #Caso a juncao seja permitida junto os dois conjuntos
            #slide 27, linha 7
            set_to_join = list(f1.union(f2))
            #Apenas um double check para evitar duplicidades no conjunto de candidatos
            if set_to_join not in Ck and len(set_to_join) == len(f1)+1:
                #Append na nova lista de candidatos possiveis
                Ck.append(set_to_join)
                #Passo de Poda
                #Determina se todos os subsets de Ck estao na lista original de pares
                #caso contrario remove da lista
                for subset_ck in gera_combinacao(set_to_join, k=len(todos_os_pares[0][0])):
                    if subset_ck not in merge_lists(todos_os_pares):
                        Ck.remove(set_to_join)
                        break
                        
    return Ck

In [250]:
#primeira iteracao
candidatos = itens_distintos(df_sells,SUP_THRESHOLD)
Fk = []
#Enquanto tivermos possiveis candidatos
while candidatos:
    print(len(candidatos))
    #Gera candidatos 
    candidatos = candidate_gen(candidatos)
    #Apenas candidatos com suporte acima do threshold sao selecionados
    valid_candidatos = gera_suporte_combinacoes(df_sells,candidatos,SUP_THRESHOLD)
    #Append na lista final de items frequentes com alto suporte
    Fk.append(valid_candidatos)
    #Atualiza candidatos para os novos serem gerados com base nesses
    candidatos = valid_candidatos


48
171
3


In [251]:
Fk

[[['318', '356'],
  ['318', '296'],
  ['318', '593'],
  ['318', '260'],
  ['318', '2571'],
  ['318', '527'],
  ['318', '480'],
  ['318', '1196'],
  ['318', '1'],
  ['318', '608'],
  ['318', '589'],
  ['318', '2858'],
  ['318', '1198'],
  ['318', '1270'],
  ['318', '110'],
  ['318', '50'],
  ['318', '457'],
  ['318', '47'],
  ['318', '150'],
  ['318', '364'],
  ['318', '590'],
  ['356', '296'],
  ['356', '593'],
  ['356', '260'],
  ['356', '2571'],
  ['356', '527'],
  ['356', '480'],
  ['356', '1196'],
  ['356', '1'],
  ['356', '589'],
  ['356', '2858'],
  ['356', '1198'],
  ['356', '1270'],
  ['356', '110'],
  ['356', '1210'],
  ['356', '50'],
  ['356', '457'],
  ['356', '4993'],
  ['356', '47'],
  ['356', '588'],
  ['356', '150'],
  ['356', '5952'],
  ['356', '364'],
  ['356', '2028'],
  ['356', '590'],
  ['356', '380'],
  ['356', '1580'],
  ['356', '592'],
  ['356', '377'],
  ['296', '593'],
  ['296', '260'],
  ['296', '2571'],
  ['296', '527'],
  ['296', '480'],
  ['296', '1196'],
 

In [252]:
def calc_confidence(I, s):
    sup_s = gera_suporte_combinacoes(df_sells,[list(s)],SUP_THRESHOLD, create_rules=True)
    sup_i = gera_suporte_combinacoes(df_sells,[list(I)],SUP_THRESHOLD, create_rules=True)
    print
    return (sup_i*1.0) /sup_s
    

In [253]:
def generate_rules(fk, CONF_TRESHOLD):
    final_rules = []
    for L in range(1, len(fk)):
        for subset in itertools.combinations(fk, L):
            I = set(fk)
            s = set(subset)
            confidence = calc_confidence(I,s)
            if confidence > CONF_TRESHOLD:
                rule = str('{} => {} = {}'.format(s, I-s, confidence))
                final_rules.append(rule)
    return final_rules
            

In [254]:
all_rules = []
for item in merge_lists(Fk):
    all_rules.append(generate_rules(item,CONF_TRESHOLD))

for item in merge_lists(all_rules):
    print(item)

{'318'} => {'356'} = 0.743801652892562
{'318'} => {'296'} = 0.7727272727272727
{'296'} => {'318'} = 0.722007722007722
{'318'} => {'593'} = 0.7396694214876033
{'593'} => {'318'} = 0.7489539748953975
{'527'} => {'318'} = 0.7301587301587302
{'1'} => {'318'} = 0.7094972067039107
{'608'} => {'318'} = 0.7450980392156863
{'2858'} => {'318'} = 0.7341772151898733
{'50'} => {'318'} = 0.7724550898203593
{'457'} => {'318'} = 0.7098445595854923
{'47'} => {'318'} = 0.7352941176470588
{'150'} => {'318'} = 0.7309941520467836
{'590'} => {'318'} = 0.7388535031847133
{'356'} => {'296'} = 0.7293233082706767
{'296'} => {'356'} = 0.749034749034749
{'593'} => {'356'} = 0.7615062761506276
{'260'} => {'356'} = 0.7072072072072073
{'2571'} => {'356'} = 0.7169811320754716
{'527'} => {'356'} = 0.7777777777777778
{'480'} => {'356'} = 0.8055555555555555
{'1196'} => {'356'} = 0.7061855670103093
{'1'} => {'356'} = 0.7486033519553074
{'589'} => {'356'} = 0.7799999999999999
{'2858'} => {'356'} = 0.7468354430379747
{'127